# 🔪 SEGMENTAÇÃO EM CHUNKS - Etapa 4/6

## 📋 O que este notebook faz

Este notebook **divide documentos em pedaços menores** (chunks) otimizados para embeddings:

- 📖 **Lê documentos** processados em `pipeline-data/processed/`
- ✂️ **Segmenta texto** usando `RecursiveCharacterTextSplitter`
- 🔗 **Aplica overlapping** (100 chars) para manter contexto entre chunks
- 🏷️ **Adiciona metadados** (documento origem, índice, tamanho)
- 💾 **Salva em JSONL** para processamento eficiente

## ⚙️ Configuração de chunking

- **Tamanho**: 500 caracteres por chunk
- **Overlap**: 100 caracteres entre chunks adjacentes  
- **Separadores**: Prioriza quebras semânticas (`\n\n`, `\n`, `. `, ` `)

## 📊 Output esperado

~322 chunks salvos em `pipeline-data/chunks/chunks.jsonl` com média de 350 caracteres cada.

---

In [1]:
import os
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json

# Configuração
CHUNK_SIZE = int(os.getenv("CHUNK_SIZE", "500"))
CHUNK_OVERLAP = int(os.getenv("CHUNK_OVERLAP", "100"))

# Diretórios
processed_dir = Path("pipeline-data/processed")
chunks_dir = Path("pipeline-data/chunks")
chunks_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório chunks recursivamente
for f in chunks_dir.rglob("*"):
    if f.is_file():
        f.unlink()

print(f"Chunk size: {CHUNK_SIZE} tokens")
print(f"Overlap: {CHUNK_OVERLAP} tokens")

# Listar documentos processados recursivamente com caminhos relativos
documents = [str(doc.relative_to(processed_dir)) for doc in processed_dir.rglob("*.md")]

print(f"Documentos encontrados: {len(documents)}")

for i, doc in enumerate(documents[:10]):
    print(f"  {doc}")

if len(documents) > 10:
    print(f"  ... e mais {len(documents) - 10} documentos")

Chunk size: 500 tokens
Overlap: 100 tokens
Documentos encontrados: 31
  30-Aprovados/Mapas/Visão Geral do Self Checkout.md
  30-Aprovados/Mapas/Visão Geral do NIC.md
  30-Aprovados/Mapas/Processa Sistemas.md
  30-Aprovados/Tópicos/Cancelamento de cupom.md
  30-Aprovados/Tópicos/Solicitação de ajuda.md
  30-Aprovados/Tópicos/Histórico de atualizações Self Checkout.md
  30-Aprovados/Tópicos/Funcionalidade do bloqueio.md
  30-Aprovados/Tópicos/Aplicação de desconto por item.md
  30-Aprovados/Tópicos/Propósito do NIC.md
  30-Aprovados/Tópicos/Reimpressão do último cupom.md
  ... e mais 21 documentos


import os
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json
import time
from datetime import datetime

# Marcar início da execução
stage_start = time.time()
start_timestamp = datetime.now().isoformat() + "Z"

# Configuração
CHUNK_SIZE = int(os.getenv("CHUNK_SIZE", "500"))
CHUNK_OVERLAP = int(os.getenv("CHUNK_OVERLAP", "100"))

# Diretórios
processed_dir = Path("pipeline-data/processed")
chunks_dir = Path("pipeline-data/chunks")
chunks_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório chunks recursivamente
for f in chunks_dir.rglob("*"):
    if f.is_file():
        f.unlink()

print(f"Chunk size: {CHUNK_SIZE} tokens")
print(f"Overlap: {CHUNK_OVERLAP} tokens")

# Listar documentos processados recursivamente com caminhos relativos
documents = [str(doc.relative_to(processed_dir)) for doc in processed_dir.rglob("*.md")]

print(f"Documentos encontrados: {len(documents)}")

for i, doc in enumerate(documents[:10]):
    print(f"  {doc}")

if len(documents) > 10:
    print(f"  ... e mais {len(documents) - 10} documentos")

In [2]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

print(f"✅ Text splitter configurado:")
print(f"  Chunk size: {CHUNK_SIZE}")
print(f"  Overlap: {CHUNK_OVERLAP}")
print(f"  Separadores: {text_splitter._separators}")

✅ Text splitter configurado:
  Chunk size: 500
  Overlap: 100
  Separadores: ['\n\n', '\n', '. ', ' ', '']


## 🔪 Segmentação de Documentos

In [3]:
# Processar cada arquivo
all_chunks = []
chunk_id = 0

for text_file in documents:
    try:
        # Construir caminho completo para leitura
        full_path = processed_dir / text_file
        
        # Ler texto
        with open(full_path, "r", encoding="utf-8") as f:
            text_content = f.read()
        
        if not text_content.strip():
            print(f"  ⚠️ Arquivo vazio: {Path(text_file).name}")
            continue
        
        # Dividir em chunks
        chunks = text_splitter.split_text(text_content)
        
        # Processar cada chunk
        for i, chunk_text in enumerate(chunks):
            chunk_data = {
                "chunk_id": chunk_id,
                "source_document": text_file.replace(".md", ""),  # Usa caminho relativo sem extensão
                "chunk_index": i,
                "text": chunk_text.strip(),
                "char_count": len(chunk_text)
            }
            
            # Adicionar ao array com estrutura mais clara
            all_chunks.append(chunk_data)
            
            chunk_id += 1
        
        print(f"  ✅ {len(chunks)} chunks criados para: {text_file}")
        
    except Exception as e:
        print(f"  ❌ Erro: {str(e)}")

print(f"\n📊 Total de chunks: {len(all_chunks)}")

  ✅ 51 chunks criados para: 30-Aprovados/Mapas/Visão Geral do Self Checkout.md
  ✅ 15 chunks criados para: 30-Aprovados/Mapas/Visão Geral do NIC.md
  ✅ 13 chunks criados para: 30-Aprovados/Mapas/Processa Sistemas.md
  ✅ 7 chunks criados para: 30-Aprovados/Tópicos/Cancelamento de cupom.md
  ✅ 6 chunks criados para: 30-Aprovados/Tópicos/Solicitação de ajuda.md
  ✅ 5 chunks criados para: 30-Aprovados/Tópicos/Histórico de atualizações Self Checkout.md
  ✅ 11 chunks criados para: 30-Aprovados/Tópicos/Funcionalidade do bloqueio.md
  ✅ 7 chunks criados para: 30-Aprovados/Tópicos/Aplicação de desconto por item.md
  ✅ 12 chunks criados para: 30-Aprovados/Tópicos/Propósito do NIC.md
  ✅ 4 chunks criados para: 30-Aprovados/Tópicos/Reimpressão do último cupom.md
  ✅ 8 chunks criados para: 30-Aprovados/Tópicos/Função do Chat NIC.md
  ✅ 9 chunks criados para: 30-Aprovados/Tópicos/Acesso ao menu do fiscal.md
  ✅ 10 chunks criados para: 30-Aprovados/Tópicos/Cronograma e marcos do projeto.md
  ✅ 7 chun

## 💾 Armazenamento e Estatísticas

In [4]:
# Salvar todos os chunks em um único arquivo JSONL
jsonl_file = chunks_dir / "chunks.jsonl"
with open(jsonl_file, "w", encoding="utf-8") as f:
    for chunk in all_chunks:
        f.write(json.dumps(chunk, ensure_ascii=False) + "\n")

print(f"\n💾 Chunks salvos em: {jsonl_file}")
print(f"📊 Total de chunks no arquivo: {len(all_chunks)}")

# Estatísticas
if all_chunks:
    avg_chars = sum(chunk["char_count"] for chunk in all_chunks) / len(all_chunks)
    max_chars = max(chunk["char_count"] for chunk in all_chunks)
    min_chars = min(chunk["char_count"] for chunk in all_chunks)
    
    print(f"\n📈 Estatísticas:")
    print(f"  Total chunks: {len(all_chunks)}")
    print(f"  Tamanho médio: {avg_chars:.0f} caracteres")
    print(f"  Tamanho mínimo: {min_chars} caracteres")
    print(f"  Tamanho máximo: {max_chars} caracteres")
    
    # Mostrar distribuição por arquivo
    from collections import Counter
    
    print(f"\n📁 Distribuição de chunks por arquivo:")
    # Usar source_document que já está no chunk
    file_counter = Counter(chunk["source_document"] for chunk in all_chunks)
    
    # Mostrar top 10 arquivos com mais chunks
    for file, count in file_counter.most_common(10):
        print(f"  {file}: {count} chunks")
    
    if len(file_counter) > 10:
        print(f"  ... e mais {len(file_counter) - 10} arquivos")


💾 Chunks salvos em: pipeline-data/chunks/chunks.jsonl
📊 Total de chunks no arquivo: 322

📈 Estatísticas:
  Total chunks: 322
  Tamanho médio: 350 caracteres
  Tamanho mínimo: 13 caracteres
  Tamanho máximo: 500 caracteres

📁 Distribuição de chunks por arquivo:
  30-Aprovados/Mapas/Visão Geral do Self Checkout: 51 chunks
  30-Aprovados/Tópicos/Componentes principais do sistema: 19 chunks
  30-Aprovados/Tópicos/Pré-requisitos técnicos: 17 chunks
  30-Aprovados/Mapas/Visão Geral do NIC: 15 chunks
  30-Aprovados/Tópicos/Padrões de Documentação do NIC: 15 chunks
  30-Aprovados/Mapas/Processa Sistemas: 13 chunks
  30-Aprovados/Tópicos/Propósito do NIC: 12 chunks
  30-Aprovados/Tópicos/Finalidade e visão do NIC: 12 chunks
  30-Aprovados/Tópicos/Funcionalidade do bloqueio: 11 chunks
  30-Aprovados/Tópicos/Cronograma e marcos do projeto: 10 chunks
  ... e mais 21 arquivos


## 📊 Relatório de Execução

In [5]:
import time
from datetime import datetime

# Se não tiver stage_start, calcular agora
if 'stage_start' not in locals():
    stage_start = time.time()
    start_timestamp = datetime.now().isoformat() + "Z"

# Calcular duração
stage_duration = time.time() - stage_start

# Carregar relatório existente
report_path = Path("pipeline-data/report.json")
if report_path.exists():
    with open(report_path, "r") as f:
        report = json.load(f)
else:
    report = {"stages": [], "context": {}, "summary": {}}

# Atualizar contexto com configurações de chunking
report["context"].update({
    "chunk_size": CHUNK_SIZE,
    "chunk_overlap": CHUNK_OVERLAP
})

# Adicionar informações desta etapa
stage_report = {
    "stage": 4,
    "name": "Segmentação em Chunks",
    "status": "SUCCESS" if len(all_chunks) > 0 else "FAILED",
    "start_time": start_timestamp if 'start_timestamp' in locals() else datetime.now().isoformat() + "Z",
    "duration_seconds": round(stage_duration, 2),
    "results": {
        "documents_processed": len(documents),
        "total_chunks": len(all_chunks),
        "avg_chunk_size": round(avg_chars) if all_chunks else 0,
        "min_chunk_size": min_chars if all_chunks else 0,
        "max_chunk_size": max_chars if all_chunks else 0
    }
}

# Adicionar ou atualizar stage no relatório
stages_updated = False
for i, stage in enumerate(report["stages"]):
    if stage["stage"] == 4:
        report["stages"][i] = stage_report
        stages_updated = True
        break

if not stages_updated:
    report["stages"].append(stage_report)

# Atualizar timestamp
report["summary"]["last_update"] = datetime.now().isoformat() + "Z"

# Salvar relatório atualizado
with open(report_path, "w") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"\n📊 Relatório atualizado: {report_path}")
print(f"⏱️ Duração da etapa: {stage_duration:.2f}s")


📊 Relatório atualizado: pipeline-data/report.json
⏱️ Duração da etapa: 0.00s
